# Урок 5. Рекуррентные нейронные сети


1. Попробуйте починить сеть по словам.

2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения


In [ ]:
from google.colab import files

train_data = 'train_data.txt'
uploaded = files.upload_file(train_data) # Загружаем файл train_data.txt

In [3]:
import re
import keras
import numpy as np
from keras.layers import *
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer

In [ ]:
with open(train_data, 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [ ]:
text

In [ ]:
len(text)

In [ ]:
num_characters = 34 #33 буквы + пробел

In [ ]:
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [ ]:
tokenizer.fit_on_texts(text)

In [ ]:
print(tokenizer.word_index)

In [ ]:
inp_chars = 6 #
data = tokenizer.texts_to_matrix(text)

In [ ]:
data[1]

In [ ]:
n = data.shape[0]-inp_chars
n  #размер обучающего множества

In [ ]:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [ ]:
len(X)

In [ ]:
X[0]

In [ ]:
Y[0]

In [ ]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)

In [ ]:
def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding

    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += d # дописываем строку

  return inp_str

In [ ]:
res = buildPhrase("утренн")
print(res)

# Слова


In [ ]:
with open(train_data, 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [ ]:
# Параметры токенизации и модели
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
total_words = len(tokenizer.word_index) + 1  # +1 для OOV токена

In [ ]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

In [ ]:
data = tokenizer.texts_to_sequences([texts])

In [ ]:
data

In [ ]:
len(data[0])

In [ ]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

In [ ]:
inp_words = 3
n = res.shape[0]-inp_words

In [ ]:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [ ]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=19)

In [ ]:
def buildPhrase(texts, str_len = 20):
  res = texts
  dataa = tokenizer.texts_to_sequences([texts])[0]
  if len(dataa) < inp_words:  # Проверяем, достаточно ли слов для начала генерации
        return "Недостаточно данных для генерации"
  for i in range(str_len):
    x = keras.utils.to_categorical(dataa[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
    inp = x.reshape(1, inp_words, maxWordsCount)
    pred = model.predict(inp) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
    res += " " + tokenizer.index_word[indx] # дописываем строку
  return res


In [ ]:
for i in range(1):
    x = keras.utils.to_categorical(data[i: i+inp_words], num_classes=maxWordsCount)

In [ ]:
x.shape

In [ ]:
res = buildPhrase("позитив добавляет годы")
print(res)

Возьмем больше данных

In [ ]:
from google.colab import files

train_data = 'train_data_new.txt'
uploaded = files.upload_file(train_data) # Загружаем файл train_data.txt

In [ ]:
with open(train_data, 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ
len(texts)

In [ ]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
dist = list(tokenizer.word_counts.items())
print(dist[:10])

In [ ]:
data = tokenizer.texts_to_sequences([texts])
len(data[0])

In [ ]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

In [ ]:
inp_words = 3
n = res.shape[0]-inp_words
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [ ]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=10)

In [ ]:
def buildPhrase(texts, str_len = 20):
  res = texts
  dataa = tokenizer.texts_to_sequences([texts])[0]
  if len(dataa) < inp_words:  # Проверяем, достаточно ли слов для начала генерации
        return "Недостаточно данных для генерации"
  for i in range(str_len):
    x = keras.utils.to_categorical(dataa[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
    inp = x.reshape(1, inp_words, maxWordsCount)
    pred = model.predict(inp) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
    res += " " + tokenizer.index_word[indx] # дописываем строку
  return res

In [ ]:
res = buildPhrase("позитив добавляет годы")
print(res)

In [ ]:
from google.colab import files

train_data = 'train_data_new.txt'
uploaded = files.upload_file(train_data)

In [ ]:
from google.colab import drive

# Подключить Google Drive
drive.mount('/content/drive')

# Для получения более осмысленного текста можно попробовать следующие подходы:

** - Увеличение размера вложенного слоя (embedding) для улучшения представления слов.
- Использование более глубокой архитектуры LSTM для более долгосрочного запоминания зависимостей в тексте.
- Увеличение количества нейронов в слоях LSTM для увеличения емкости модели.
- Использование Dropout для регуляризации и предотвращения переобучения. **

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Загрузка данных
with open('your_text_file.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')

# Параметры токенизации и модели
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
total_words = len(tokenizer.word_index) + 1  # +1 для OOV токена

# Подготовка последовательностей данных для обучения
sequences = tokenizer.texts_to_sequences([texts])[0]
input_sequences = []
for i in range(1, len(sequences)):
    n_gram_sequence = sequences[:i+1]
    input_sequences.append(n_gram_sequence)

# Дополнение последовательностей для обеспечения одинаковой длины
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Создание предикторов и меток
X, labels = input_sequences[:,:-1], input_sequences[:,-1]
Y = to_categorical(labels, num_classes=total_words)

In [ ]:
# Построение модели
model = Sequential()
model.add(Embedding(total_words, 300, input_length=max_sequence_len-1))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.summary()

In [ ]:
# Обучение модели
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

history = model.fit(X, Y, epochs=50, verbose=1, batch_size=128, callbacks=[early_stop, reduce_lr])

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/model10.h5')

In [ ]:
# Генерация текста
def generate_text(seed_text, model, max_sequence_len, next_words=50):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predictions = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predictions, axis=-1)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

seed_text = 'в начале было слово'
generated_text = generate_text(seed_text, model, max_sequence_len)
print(generated_text)

** Этот код увеличивает размер вложенного слоя (embedding) до 300 и количество нейронов в LSTM слоях до 256 и 128 соответственно.
Dropout используется для регуляризации модели. Обучение проводится на протяжении 50 эпох с ранней остановкой и уменьшением learning rate при необходимости. **

In [ ]:
%pip install transformers[torch]
%pip install accelerate -U

In [ ]:
import os
os._exit(00)

In [ ]:
from google.colab import drive

# Подключить Google Drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

train_data = 'train_data_new.txt'
uploaded = files.upload_file(train_data) # Загружаем файл train_data.txt

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Загрузка токенизатора и модели
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Подготовка данных для обучения
train_dataset = TextDataset(
  tokenizer=tokenizer,
  file_path='train_data_new.txt', # Путь к файлу с данными для обучения
  block_size=128)

data_collator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import TrainingArguments

# Настройка параметров обучения
training_args = TrainingArguments(
  output_dir='./gpt2-finetuned',
  overwrite_output_dir=True,
  num_train_epochs=2,
  per_device_train_batch_size=4,
  save_steps=10_000,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

# Создание объекта Trainer и запуск обучения
trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=data_collator,
  train_dataset=train_dataset,
)

trainer.train()

# Сохранение дообученной модели
model.save_pretrained('/content/drive/MyDrive/gpt2')

In [ ]:
def generate_text(prompt, model_path='./gpt2-finetuned', max_length=50, temperature=0.7):

    # Преобразование входной фразы в токены
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Генерация текста
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        num_return_sequences=1
    )

    # Декодирование сгенерированного текста
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

prompt = 'позитивный человек'
generated_text = generate_text(prompt)
print(generated_text)

In [ ]:
prompt = 'изменить взгляд на жизнь'
generated_text = generate_text(prompt)
print(generated_text)